# Machine learning bank's credit card service churn prediction
In this project, I will build a <b>machine learning model to predict customer churn in a bank's credit card service.</b> It involves <b>supervised learning (using a labeled training set) for classification</b>, where the <b>target</b> is <b>1</b> if the <b>customer churned</b>, else <b>0.</b>

I will use the following <b>pipeline</b> based on <b>CRISP-DM framework</b>

<b>1. Define the business problem.</b><br>
<b>2. Collect the data and get a general overview of it.</b><br>
<b>3. Split the data into train and test sets.</b><br>
<b>4. Explore the data (exploratory data analysis)</b><br>
<b>5. Data cleaning and preprocessing.</b><br>
<b>6. Model training, comparison, selection and tuning.</b><br>
<b>7. Final production model testing and evaluation.</b><br>
<b>8. Conclude and interpret the model results.</b><br>
<b>9. Deploy.</b><br>

In <b>this notebook</b>, I will perform <b>machine learning modelling, covering steps 5 to 8 of the pipeline above.</b> The main <b>objective</b> here is to <b>build a model that can predict as many as possible churn customers, thus recall is the metric of interest.</b> Once I build this model, the <b>bank can plan strategies for client retention, maintaining healthy revenue. Acquiring new customers is costlier than keeping existing ones.</b> Furthermore, I will approach these steps in detail below, explaining why I am making each decision.